# DEMO - Training and testing HerdNet on nadir aerial images

## Installations

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install the dependencies
# !pip install albumentations==1.0.3
# !pip install fiftyone==0.14.3
# !pip install hydra-core==1.1.0
# !pip install opencv-python==4.5.1.48
# !pip install pandas==1.2.3
# !pip install pillow==8.2.0
# !pip install scikit-image==0.18.1
# !pip install scikit-learn==1.0.2
# !pip install scipy==1.6.2
# !pip install wandb==0.10.33

In [4]:
# Download and install the code
import sys

!git clone https://github.com/Alexandre-Delplanque/HerdNet
!cd '/content/HerdNet' && python setup.py install

sys.path.append('/content/HerdNet')

Cloning into 'HerdNet'...
remote: Enumerating objects: 570, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 570 (delta 149), reused 132 (delta 132), pack-reused 377 (from 2)
Receiving objects: 100% (570/570), 205.52 KiB | 2.74 MiB/s, done.
Resolving deltas: 100% (370/370), done.
running install
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_di

## Create datasets

In [2]:
# Download some of the data of Delplanque et al. (2021) as an example
!gdown 1KfGBYaXo9LVOiz9qO1pxRH3ZC5bsywCW -O /content/data.zip
!unzip -oq /content/data.zip -d /content


Downloading...
From (original): https://drive.google.com/uc?id=1KfGBYaXo9LVOiz9qO1pxRH3ZC5bsywCW
From (redirected): https://drive.google.com/uc?id=1KfGBYaXo9LVOiz9qO1pxRH3ZC5bsywCW&confirm=t&uuid=13b3eb82-62b1-4ccc-8333-dd7ff7f42bbe
To: /content/data.zip
100% 11.6G/11.6G [01:29<00:00, 128MB/s]


In [5]:
# Set the seed
from animaloc.utils.seed import set_seed

set_seed(9292)

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [6]:
# Create validation patches using the patcher tool (for demo)
from animaloc.utils.useful_funcs import mkdir

mkdir('/content/data/val_patches')
!python /content/HerdNet/tools/patcher.py /content/data/val 512 512 0 /content/data/val_patches -csv /content/data/val.csv -min 0.0 -all False

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
Creating the buffer: 100% 111/111 [00:44<00:00,  2.52it/s]
Exporting patches: 100% 111/111 [02:08<00:00,  1.16s/it]


In [20]:
# Training, validation and test datasets
import albumentations as A

from animaloc.datasets import CSVDataset
from animaloc.data.transforms import MultiTransformsWrapper, DownSample, PointsToMask, FIDT

patch_size = 512
num_classes = 7
down_ratio = 2

# train_dataset = CSVDataset(
#     csv_file = '/content/data/train_patches.csv',
#     root_dir = '/content/data/train_patches',
#     albu_transforms = [
#         A.VerticalFlip(p=0.5),
#         A.HorizontalFlip(p=0.5),
#         A.RandomRotate90(p=0.5),
#         A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.2),
#         A.Blur(blur_limit=15, p=0.2),
#         A.Normalize(p=1.0)
#         ],
#     end_transforms = [MultiTransformsWrapper([
#         FIDT(num_classes=num_classes, down_ratio=down_ratio),
#         PointsToMask(radius=2, num_classes=num_classes, squeeze=True, down_ratio=int(patch_size//16))
#         ])]
#     )

val_dataset = CSVDataset(
    csv_file = '/content/data/val_patches/gt.csv',
    root_dir = '/content/data/val_patches',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

test_dataset = CSVDataset(
    csv_file = '/content/data/test.csv',
    root_dir = '/content/data/test',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

In [21]:
# Dataloaders
from torch.utils.data import DataLoader

# train_dataloader = DataLoader(dataset = train_dataset, batch_size = 4, shuffle = True)

val_dataloader = DataLoader(dataset = val_dataset, batch_size = 1, shuffle = False)

test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, shuffle = False)

## Define HerdNet for training

In [11]:
from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

herdnet = HerdNet(num_classes=num_classes, down_ratio=down_ratio, pretrained=False).cuda()

weight = Tensor([0.1, 1.0, 2.0, 1.0, 6.0, 12.0, 1.0]).cuda()

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean', weight=weight), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
    ]

herdnet = LossWrapper(herdnet, losses=losses)

## Create the Trainer

In [22]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

work_dir = '/content/output'
mkdir(work_dir)

lr = 1e-4
weight_decay = 1e-3
epochs = 100

optimizer = Adam(params=herdnet.parameters(), lr=lr, weight_decay=weight_decay)

metrics = PointsMetrics(radius=20, num_classes=num_classes)

stitcher = HerdNetStitcher(
    model=herdnet,
    size=(patch_size,patch_size),
    overlap=160,
    down_ratio=down_ratio,
    reduction='mean'
    )

evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=val_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=work_dir,
    header='validation'
    )

# trainer = Trainer(
#     model=herdnet,
#     train_dataloader=train_dataloader,
#     optimizer=optimizer,
#     num_epochs=epochs,
#     evaluator=evaluator,
#     work_dir=work_dir
#     )

## Start training

In [ ]:
trainer.start(warmup_iters=100, checkpoints='best', select='max', validate_on='f1_score')

## Test the model

In [12]:
# Path to your .pth file
import gdown

pth_path = ''

if not pth_path:
    gdown.download(
        'https://drive.google.com/uc?export=download&id=1-WUnBC4BJMVkNvRqalF_HzA1_pRkQTI_',
        '/content/20220413_herdnet_model.pth'
        )
    pth_path = '/content/20220413_herdnet_model.pth'

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1-WUnBC4BJMVkNvRqalF_HzA1_pRkQTI_
From (redirected): https://drive.google.com/uc?export=download&id=1-WUnBC4BJMVkNvRqalF_HzA1_pRkQTI_&confirm=t&uuid=becfebfa-1bdb-4748-b772-9c2a914e66bc
To: /content/20220413_herdnet_model.pth
100%|██████████| 220M/220M [00:01<00:00, 152MB/s]


In [15]:
# Create output folder
test_dir = '/content/test_output'
mkdir(test_dir)

In [16]:
# Load trained parameters
from animaloc.models import load_model

herdnet = load_model(herdnet, pth_path=pth_path)

In [23]:
# Create an Evaluator
test_evaluator = HerdNetEvaluator(
    model=herdnet,
    dataloader=test_dataloader,
    metrics=metrics,
    stitcher=stitcher,
    work_dir=test_dir,
    header='test'
    )

In [24]:
# Start testing
test_f1_score = test_evaluator.evaluate(returns='f1_score')

/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
INFO:20250516_evaluation:test [  1/258] eta: 0:20:49 n: 1 recall: 1.0 precision: 0.2 f1-score: 0.33 MAE: 4.0 MSE: 16.0 RMSE: 4.0 time: 4.8446 data: 0.6923 max mem: 1471


test [  1/258] eta: 0:20:49 n: 1 recall: 1.0 precision: 0.2 f1-score: 0.33 MAE: 4.0 MSE: 16.0 RMSE: 4.0 time: 4.8446 data: 0.6923 max mem: 1471


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

test [ 11/258] eta: 0:14:30 n: 1 recall: 1.0 precision: 1.0 f1-score: 1.0 MAE: 0.0 MSE: 0.0 RMSE: 0.0 time: 3.5096 data: 0.6267 max mem: 1471


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

...

test [241/258] eta: 0:00:57 n: 13 recall: 0.46 precision: 0.86 f1-score: 0.6 MAE: 6.0 MSE: 36.0 RMSE: 6.0 time: 2.8580 data: 0.5284 max mem: 1471


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

test [251/258] eta: 0:00:25 n: 2 recall: 1.0 precision: 1.0 f1-score: 1.0 MAE: 0.0 MSE: 0.0 RMSE: 0.0 time: 2.8811 data: 0.5240 max mem: 1471


/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transform to process it.
  self._set_keys()
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for keypoints, but no transf

test [258/258] eta: 0:00:03 n: 3 recall: 0.67 precision: 0.67 f1-score: 0.67 MAE: 0.0 MSE: 0.0 RMSE: 0.0 time: 2.8706 data: 0.5246 max mem: 1471
test Total time: 0:13:39 (3.1744 s / it)


In [25]:
# Print global F1 score (%)
print(f"F1 score = {test_f1_score * 100:0.0f}%")

F1 score = 83%


In [26]:
# Get the detections
detections = test_evaluator.results
detections

,class,n,recall,precision,f1_score,confusion,mae,mse,rmse,ap
0,1,675,0.875556,0.838298,0.856522,0.057416,1.718310,7.887324,2.808438,0.801938
1,2,349,0.727794,0.916968,0.811502,0.066176,3.500000,35.642857,5.970164,0.693974
2,3,477,0.903564,0.848425,0.875127,0.092632,1.028302,3.952830,1.988173,0.859655
3,4,74,0.445946,0.388235,0.415094,0.057143,2.243243,9.810811,3.132221,0.272273
4,5,36,0.694444,0.657895,0.675676,0.193548,1.230769,2.153846,1.467599,0.595822
5,6,688,0.728198,0.675202,0.700699,0.001992,2.601942,19.669903,4.435076,0.655523
6,binary,2299,0.844715,0.824628,0.834551,0.000000,1.906977,13.542636,3.680032,0.722731
